In [ ]:
"Author": "Lee Meng",
  "Category": "",
  "Date": "2018-04-28 19:00",
  "Description": "在這個大數據時代，誰能最有效率地從巨量資料中找出洞見，誰就是贏家。本篇將介紹 Presto，一個能讓資料科學家快速分析 Petabyte 等級資料量的分散式 SQL 查詢引擎。透過了解其背後的運作原理以及解決的痛點，資料科學家將能了解為何他們能使用 Presto 在最短的時間做出分析結果並創造最大價值。",
  "Image": "Tour_de_babel.jpg",
  "Image_credit_url": "https://commons.wikimedia.org/wiki/File:Tour_de_babel.jpeg",
  "Slug": "all-you-need-to-know-about-presto-as-a-data-scientist",
  "Tags": "SQL, big data, presto",
  "Title": "SQL 與 Presto - 大數據時代不可或缺的分析神器",

在這個大數據時代，誰能最有效率地從巨量資料中找出洞見，誰就是贏家。

本篇將簡單介紹 SQL 的概念、在大數據上的發展，以及一個能快速分析 [Petabyte](https://zh.wikipedia.org/wiki/%E6%8B%8D%E5%AD%97%E8%8A%82) 等級資料量的分散式 SQL 查詢引擎：[Presto](https://prestodb.io/)。資料科學家將能了解為何能利用兩者在最短的時間做出分析結果並創造最大價值。

## 目錄
- [使用 SQL 與數據對話](#使用-SQL-與數據對話)
    - [用 Python 達到 SQL 查詢效果](#用-Python-達到-SQL-查詢效果)
- [大數據上的 SQL](#大數據上的-SQL)
- [Presto 介紹](#Presto-介紹)

## 使用 SQL 與數據對話

身為資料科學家或者是分析人員，我們都知道 [結構化語言（SQL）](https://zh.wikipedia.org/wiki/SQL) 基本上是必備的分析工具。[SQL 是一種程式語言](https://zh.wikipedia.org/wiki/%E5%AE%A3%E5%91%8A%E5%BC%8F%E7%B7%A8%E7%A8%8B)。一般而言，我們可以透過它對被儲存在[關聯式資料庫](https://zh.wikipedia.org/wiki/%E5%85%B3%E7%B3%BB%E6%95%B0%E6%8D%AE%E5%BA%93)裡頭的資料進行查詢或操作。

但你可能會問一個最基本的問題：

In [ ]:
#blockquote
做為一個程式語言，為何 SQL 那麼地受到企業以及資料科學家的重視？

在沒接觸過 SQL 之前，你可能會想
- 「我們有 Python、R，不學 SQL 應該也沒關係吧？」
- 「又要學一個程式語言好麻煩。」

為了釐清這些疑問，讓我們做一個假想實驗。比方說我們現在想要知道某個特定顧客過去的所有購買記錄。

如果你熟悉 SQL 的話，可以對資料庫下一個簡單的查詢（Query）：

```sql
SELECT c.name AS customer, 
       o.totalprice, 
       o.orderdate 
  FROM customer AS c 
       INNER JOIN orders AS o 
       ON c.custkey = o.custkey 
 WHERE c.name = 'Customer#000000001' 
 ORDER BY o.orderdate;
```

上面這個查詢翻為白話就是：
- 從顧客清單 `customer` 還有購賣紀錄 `orders` 裡頭，找出名為 Customer#000000001 的顧客的所有購買紀錄，並把那些紀錄依照購買日期排序。

注意，我們並沒有告訴資料庫「如何」取得這些資料，比方說：
- 怎麼合併顧客跟購買紀錄？
- 怎麼過濾特定顧客？
- 怎麼排序？

我們只告訴它該給我們「什麼資料」。而得到的結果是：

```text

      customer            | totalprice | orderdate
--------------------+------------+------------
 Customer#000000001 |  152411.41 | 1993-06-05
 Customer#000000001 |  165928.33 | 1995-10-29
 Customer#000000001 |  270087.44 | 1997-03-04
```

如同我們預期，只有該顧客的購買紀錄被回傳，且依照購買日期 `orderdate` 從早排到晚。


實際上，資料庫可能需要做以下運算來取得資料：
- 將顧客表格 `customer` 以及購買紀錄的表格 `orders` 分別命名為 `c` 及 `o`
- 依照共通的鍵值 `custkey` 合併（`JOIN`）兩表格
- 找出特定顧客 `Customer#000000001` 的購買記錄
- 將該紀錄依照購買日期 `orderdate` 排序
- 選出要顯示的欄位 

這些運算最後都得依照「某個」順序執行，但是我們不需要考慮這些事情，完全依靠資料庫的[查詢最佳化器（Query Optimizer）](http://db.cs.berkeley.edu/papers/fntdb07-architecture.pdf)來幫我們決定。

寫 SQL 敘述時，你可以理解成我們是指定要的資料，而查詢最佳化器會依照此需求，找出一個最佳路徑來取得必要的資料。

換句話說，當我們在寫 SQL 的時候，是在進行[宣告式程式設計（Declarative Programming）](https://zh.wikipedia.org/wiki/%E5%AE%A3%E5%91%8A%E5%BC%8F%E7%B7%A8%E7%A8%8B)：我們只告訴資料庫，我們想要什麼資料（What），而不是怎麼取得（How）它們。

這跟一般常見的[命令式程式語言（Imperative Programming）](https://zh.wikipedia.org/wiki/%E6%8C%87%E4%BB%A4%E5%BC%8F%E7%B7%A8%E7%A8%8B)如 Python、Java 有所不同。在寫 SQL 時，我們告訴資料庫它該達成的目標 - 取得什麼資料（What）；在寫 Python 時，我們得告訴程式該怎麼達成該目標（How）。

為了進一步闡述這個概念，接著讓我們試著使用 Python 來取得跟上面的 SQL 查詢一樣的結果。 

### 用 Python 達到 SQL 查詢效果

首先先假設顧客資料是一個 `list`，裡頭包含多個 `dict`。每個 `dict` 則代表一個顧客的資料：

In [40]:
customers = [
    {"name": "Customer#000000001", "custkey": "1"},
    {"name": "Customer#000000002", "custkey": "2"}
]

而購買記錄則是一個 `dict`，`dict` 的鍵值為所有顧客的 `custkey`；鍵值對應的值則是包含該顧客所有購買紀錄的 `list`：

In [41]:
orders = {
    "1": [{"totalprice": 152411.41, "orderdate": "1993-06-05"},
          {"totalprice": 270087.44, "orderdate": "1997-03-04"},
          {"totalprice": 165928.33, "orderdate": "1995-10-29"}
         ]
}

所以 `orders["1"]` 就代表 `custkey = 1` 的顧客的購買紀錄。

了解背後的資料結構以後，我們可以寫一段 Python 程式碼來取得資料：

In [42]:
print("customer           | totalprice| orderdate ")
print("------------------ | ----------| --------- ")
# 從所有顧客找符合條件的人
for c in customers:
    # 跳過我們沒興趣的顧客
    if c['name'] != 'Customer#000000001':
        continue
    # 利用 custkey 取德該顧客的購買紀錄
    c_orders = orders[c['custkey']]
    
    # 依照 orderdate 排序購買紀錄
    c_orders_sorted = sorted(c_orders, key=lambda x: x['orderdate'])
    
    # 將所有排序後的記錄回傳
    for o in c_orders_sorted:
        values = [c['name'], str(o['totalprice']), str(o['orderdate'])]
        print(" | ".join(values))
    # 已經找到該顧客，提早結束迴圈以減少處理時間
    break

customer           | totalprice| orderdate 
------------------ | ----------| --------- 
Customer#000000001 | 152411.41 | 1993-06-05
Customer#000000001 | 165928.33 | 1995-10-29
Customer#000000001 | 270087.44 | 1997-03-04


所以我們使用 Python 達到跟上面的 SQL 查詢一樣的結果了。但兩者在執行上有什麼差異？

使用命令式程式語言來處理資料時，我們需要：
- 了解資料結構（顧客是存在 `list` 還是 `dict` ？）
- 明確地定義執行步驟（先排序購買記錄 `orders` 還是先把顧客 `customers` 跟購買紀錄合併？）
- 最佳化（如最後的 `break` ）

再看一次先前的 SQL 查詢（+註解）：

```sql
-- 給我以下幾個欄位：顧客名稱、總購買金額、購買日期
SELECT c.name AS customer,
       o.totalprice,
       o.orderdate
-- 將有相同 custkey 的顧客跟購買紀錄合併
  FROM customer AS c
       INNER JOIN orders AS o
       ON c.custkey = o.custkey
-- 只需要此顧客的購買紀錄
 WHERE c.name = 'Customer#000000001'
-- 依照購買日期排序
 ORDER BY o.orderdate;
``` 

這裡頭我們不需要了解資料庫是怎麼被實作的，不需要了解資料被以什麼形式儲存，也不需要定義要以什麼順序執行查詢，更不用做最佳化。這些事情全部交給背後的資料庫處理，使得資料科學家可以專注在更高層次的問題：「我們需要什麼資料？」

而這正是 SQL 最強大的地方。

In [ ]:
#blockquote
SQL 讓資料科學家可以專注在需要「什麼」資料而非要「怎麼」取得。

## 大數據上的 SQL

正是因為 SQL 有如此特性，使得它成為[ 2018 前幾受歡迎的程式語言](https://insights.stackoverflow.com/survey/2018/)。

但是資料並不都只存在關聯式資料庫裡頭：很多企業會將大量的資料存在 [Hadoop 分散式檔案系統（HDFS）](https://zh.wikipedia.org/wiki/Apache_Hadoop) 或者是 [AWS S3](https://aws.amazon.com/tw/s3/) 以供快速的資料處理。這邊的資料處理常使用知名的 [MapReduce](https://zh.wikipedia.org/wiki/MapReduce) 架構。


## Presto

In [ ]:
非結構化資料，大量資料 -> Hadoop / HDFS
SQL-on-Hadoop
MPP
Presto 的發跡
Presto 介紹

運作原理
解決的痛點
資料不用移轉 (vs MPP)
不用白費力氣的中繼存檔, in-memory (vs Hadoop)
類似資料湖
你為何會想用它
Presto 實際操作
結語
帥氣的標題：巴比倫塔
ANSI SQL 是資料科學家必備的程式語言能力之一，跟資料工程師重複的部分
如果你想要用更高層次思考問題
現在就開始投資吧！